In [ ]:
!pip install GPUtil

import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU not available.")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q -i https://pypi.org/simple/ bitsandbytes
!pip install -q transformers
!pip install -q datasets

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig,LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

if 'COLAB_GPU' in os.environ:
    from google.colab import output
    output.enable_custom_widget_manager()

In [ ]:
!huggingface-cli login

In [ ]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                            quantization_config=bnb_config)

In [ ]:
train_dataset = load_dataset("csv", data_files="dataset.csv", split="train")
train_dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=False,
                                           add_eos_token=True)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'],padding="max_length", truncation=True)
tokenized_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_dataset

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    # rank of the update matrices
    # Lower rank results in smaller matrices with fewer trainable params
    r=8,

    # impacts low-rank approximation aggressiveness
    # increasing value speeds up training
    lora_alpha=64,

    # modules to apply the LoRA update matrices
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "down_proj",
        "up_proj",
        "o_proj"
    ],

    # determines LoRA bias type, influencing training dynamics
    bias="none",

    # regulates model regularization; increasing may lead to underfitting
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [ ]:
trainer = transformers.Trainer(
    model=model, # Mô hình sử dụng tinh chỉnh
    train_dataset=tokenized_dataset, # Bộ dataset sử dụng tinh chỉnh
    args=transformers.TrainingArguments(
        output_dir="./finetuned-llama-2-model-TuyenSinhPTIT2024",  # Thư mục lưu lại các checkpoint huấn luyện
        per_device_train_batch_size=2, # Số mẫu dữ liệu mỗi bath cho GPU
        gradient_accumulation_steps=2, # Tích lũy gradient sau 2 bước trước khi thực hiện cập nhật trọng số
        num_train_epochs=3, # Số lần duyệt qua toàn bộ dữ liệu huấn luyện
        learning_rate=2e-4, # Xác định mức độ thay đổi trọng số trong mỗi bước cập nhật
        bf16=False,      # Tắt định dạng bfloat16 do phần cứng không hỗ trợ
        optim="paged_adamw_8bit", # Sử dụng tối ưu hóa Adamw phiên bản 8bit để tiết kiệm bộ nhớ và tăng hiệu suất
        logging_dir="./logs",# Thư mục sử dụng để lưu thông tin huấn luyện
        save_strategy="epoch", # Lưu lại model huấn luyện sau mỗi epoch
        logging_steps = 10, # Ghi lại thông tin nhật kí huấn luyện
        report_to="none"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False # Tắt cache trong bộ huấn luyện để tối ưu bộ nhớ
trainer.train() # Tiến hành huấn luyện

trainer.save_model("./finetuned_llama2_7b_chat")
tokenizer.save_pretrained("./finetuned_llama2_7b_chat")

In [ ]:

del trainer
import gc
gc.collect()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig,LlamaTokenizer
from peft import PeftModel

base_model_id = "meta-llama/Llama-2-7b-chat-hf"

nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False,
                                           trust_remote_code=True)


# modelFinetuned = PeftModel.from_pretrained(base_model,"finetuneModel")
# modelFinetuned = modelFinetuned.merge_and_unload()

In [ ]:
### ENTER YOUR QUESTION BELOW

question = "Trả lời câu hỏi sau bằng tiếng việt: Điểm chuẩn ngành công nghệ thông tin của Học viện công nghệ bưu chính viễn thông năm 2023 tại cơ sở BVH?"

# Format the question
eval_prompt = f"{question}\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()
with torch.no_grad():
    print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024)[0], skip_special_tokens=True))
torch.cuda.empty_cache()

In [ ]:
modelFinetuned = PeftModel.from_pretrained(base_model,"newmodel")
modelFinetuned = modelFinetuned.merge_and_unload()

In [ ]:
modelFinetuned.save_pretrained("./llama-2-chat-hf-for-TuyenSinhPTIT2024")

In [ ]:
modelFinetuned.push_to_hub("llama-2-chat-hf-for-TuyenSinhPTIT2024", use_temp_dir=False)
tokenizer.push_to_hub("llama-2-chat-hf-for-TuyenSinhPTIT2024", use_temp_dir=False)